In [2]:
# This cell imports libraries and loads the CSV into a DataFrame
import pandas as pd
import numpy as np

# Read the provided CSV
file_path = 'D:/academics/data_set_BA/deutsche bank/MOCK_DATA.csv'
df_ssi = pd.read_csv(file_path, encoding='ascii')

print('Loaded rows:', len(df_ssi))
print('Columns:', list(df_ssi.columns))
print('Head preview:')
print(df_ssi.head())

Loaded rows: 1000
Columns: ['Client_Name', 'Client_ID', 'Settlement_Currency', 'Bank_Name', 'Account_Number', 'Swift_Code', 'Source_System', 'Creation_Date']
Head preview:
       Client_Name Client_ID Settlement_Currency           Bank_Name  \
0     Dov Cronchey    CLT001                 PKR     Maple Leaf Bank   
1      Carson Took    CLT002                 NGN        Emerald Bank   
2       Rob Dybald    CLT003                 EUR  Mountain View Bank   
3  Joelle McGiffie    CLT004                 IDR    Silver Lake Bank   
4    Wilt Runchman    CLT005                 CNY     Maple Leaf Bank   

   Account_Number Swift_Code Source_System Creation_Date  
0       113000609   CHASUS33       SystemD    10/31/2020  
1       101100045   BOFAUS3N       SystemG     3/18/2020  
2        91101455   WFBIUS6S       SystemF    10-08-2020  
3       242071237   PNCCUS33       SystemA     5/29/2021  
4       123006389   MRMDUS33       SystemI    01-04-2021  


In [3]:
# This cell performs the requested data cleaning and feature engineering steps

import re

# Work on a copy
clean = df_ssi.copy()

# --- 1) Identify and handle missing values for Swift_Code and Account_Number ---
# Normalize blanks and obvious placeholders to NaN
for col in ['Swift_Code', 'Account_Number']:
    if col in clean.columns:
        clean[col] = clean[col].replace(['', 'NA', 'N/A', 'null', 'None', 'nan'], np.nan)

# Ensure Account_Number is string, strip non-digits for standardization logic
clean['Account_Number'] = clean['Account_Number'].astype(str)
clean.loc[clean['Account_Number'].str.lower().isin(['nan', 'none', 'nat', 'na']), 'Account_Number'] = np.nan

# Remove non-digit characters and keep leading zeros by working in string
def only_digits(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    digits = re.sub('[^0-9]', '', s)
    return digits if digits != '' else np.nan

clean['Account_Number'] = clean['Account_Number'].apply(only_digits)

# Fill Account_Number: if missing and duplicates exist by Client_Name + Bank_Name, forward/back fill within that group
clean['Account_Number'] = clean.groupby(['Client_Name', 'Bank_Name'])['Account_Number'].transform(lambda s: s.ffill().bfill())

# As a fallback, if still missing, try to infer from other identical client records irrespective of bank
clean['Account_Number'] = clean.groupby(['Client_Name'])['Account_Number'].transform(lambda s: s.ffill().bfill())

# --- Swift code normalization ---
# Uppercase and strip spaces
clean['Swift_Code'] = clean['Swift_Code'].astype(str).str.upper().str.replace(' ', '', regex=False)
clean.loc[clean['Swift_Code'].str.lower().isin(['nan', 'none', 'na']), 'Swift_Code'] = np.nan

# Standard SWIFT/BIC length is 8 or 11. Define a cleanup function:
def normalize_swift(sw):
    if pd.isna(sw):
        return np.nan
    s = str(sw).upper().strip()
    s = re.sub('[^A-Z0-9]', '', s)
    # If length is 7 due to dropped char, cannot reliably fix -> leave NaN so we can fill later
    if len(s) in [8, 11]:
        return s
    if len(s) < 8:
        return np.nan
    if len(s) > 11:
        return s[:11]
    # If length is 9 or 10, pad to 11 with XXX (branch code placeholder)
    if len(s) in [9, 10]:
        return (s + 'X' * (11 - len(s)))[:11]
    return s

clean['Swift_Code'] = clean['Swift_Code'].apply(normalize_swift)

# Fill missing Swift_Code by mapping from Bank_Name (use most frequent known SWIFT per bank in the data)
most_common_swift_by_bank = (
    clean.dropna(subset=['Swift_Code'])
         .groupby('Bank_Name')['Swift_Code']
         .agg(lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else np.nan)
)

mask_swift_missing = clean['Swift_Code'].isna()
clean.loc[mask_swift_missing, 'Swift_Code'] = clean.loc[mask_swift_missing, 'Bank_Name'].map(most_common_swift_by_bank)

# As a second fallback, fill Swift_Code from duplicates within Client_Name groups
clean['Swift_Code'] = clean.groupby('Client_Name')['Swift_Code'].transform(lambda s: s.ffill().bfill())

# --- 2) Format standardization ---
# Account_Number: keep as string of digits, set empty to NaN, optionally enforce a max length (no truncation unless absurd)
clean.loc[clean['Account_Number'].notna() & (clean['Account_Number'].str.len() == 0), 'Account_Number'] = np.nan

# Swift_Code: ensure 8 or 11 length after prior normalization; if 8, append XXX to make 11 for uniformity in storage, but keep original in a new column
clean['Swift_Code_Original'] = df_ssi['Swift_Code']
clean['Swift_Code_Standard'] = clean['Swift_Code'].apply(lambda s: (s + 'XXX') if isinstance(s, str) and len(s) == 8 else s)

# Keep the standardized column as the working Swift_Code
clean['Swift_Code'] = clean['Swift_Code_Standard']

# --- 3) Duplicate removal ---
# Identify duplicates based on Client_Name and Account_Number; keep first occurrence
clean_before = len(clean)
clean = clean.sort_values(['Client_Name', 'Account_Number', 'Creation_Date'])
clean = clean.drop_duplicates(subset=['Client_Name', 'Account_Number'], keep='first')
clean_after = len(clean)

# --- 4) Risk Flagging ---
clean['Risk_Flag'] = np.where(clean['Source_System'].astype(str).str.strip().str.lower() == 'high risk email', 'High', 'Normal')

print('Most common SWIFT by bank computed')
print(most_common_swift_by_bank.head())
print('Rows before duplicate removal:', clean_before)
print('Rows after duplicate removal:', clean_after)
print('Head after cleaning:')
print(clean.head())

Most common SWIFT by bank computed
Bank_Name
Emerald Bank          BOFAUS3N
Golden Gate Bank      CITIUS33
Maple Leaf Bank       CITIUS33
Mountain View Bank    CITIUS33
Ocean Breeze Bank     CITIUS33
Name: Swift_Code, dtype: object
Rows before duplicate removal: 1000
Rows after duplicate removal: 1000
Head after cleaning:
         Client_Name Client_ID Settlement_Currency           Bank_Name  \
539  Abbe MacAlinden    CLT010                 CNY   Ocean Breeze Bank   
604       Abbe Swann    CLT005                 CNY  Mountain View Bank   
406     Abbey Raulin    CLT007                 IDR   Ocean Breeze Bank   
543     Abbie Cousen    CLT004                 UGX     Maple Leaf Bank   
304   Abigail Flewan    CLT005                 USD         Sunset Bank   

    Account_Number   Swift_Code Source_System Creation_Date  \
539      226071237  SVRNUS33XXX       SystemI     6/19/2020   
604       74900194  PNCCUS33XXX       SystemF     3/22/2021   
406      104902127  CITIUS33XXX       Syst